In [11]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Table
from sqlalchemy import create_engine, inspect
from sqlalchemy import MetaData
from edgerdb import helper_functions as hlp
import pandas as pd
import numpy as np


meta = MetaData()

engine = create_engine('postgresql://analyst:@localhost:5432/edgar')

messages = Table('filings', meta, autoload=True, autoload_with=engine)

def sql_to_df(query, connection, index=None):
    df = pd.read_sql(query, connection)
    connection.close()
    return df

In [28]:
cik_and_tickers = sql_to_df('select * from temp_cik_to_ticker;', engine.connect()).set_index(keys='index')

In [29]:
cik_and_tickers.head()

,cik,company_name,exchange,isactive,symbol
index,,,,,
0,1591890,"1347 PROPERTY INSURANCE HOLDINGS, INC.",NASDAQ,1,PIH
1,1508475,"21VIANET GROUP, INC.",NASDAQ,1,VNET
2,1459417,"2U, INC.",NASDAQ,1,TWOU
3,1295484,"51JOB, INC.",NASDAQ,1,JOBS
4,1295484,"51JOB, INC.",NASDAQ,1,JOBS


In [39]:
tickers = cik_and_tickers['symbol'].tolist()
